# Import the Sandbox 


<div class="alert alert-block alert-warning">
    <b> Acces to all modules: </b> This Notebook allow to change between modules using only widgets from the bokeh panel
</div>

In [ ]:
#Import the sandbox
import os,sys
sys.path.append('../../')
import sandbox as sb

## Start the Panel

The panel will start with a simple line, opening a bokeh server for the projector and the widgets. Position the window in the projector image and use the widgets to change between modules. Each module will open its respective widgets in a new bokeh server allowing an easy interaction and control over all the features.

Please always pass a correct calibration file 

In [ ]:
#path to the calibration file
CALIBRATION_FILE = '../calibration_files/my_calibration.json'

In [ ]:
sb.start_server(CALIBRATION_FILE, aruco_marker=True, geo_model=None)

## Calibration
In case that the frame of the sandbox is not calibrated, for a fast calibration you can use the following line of code and follow the bokeh instructions. In case of a more detailed explanation on how to do the calibration please go to the tutorials

In [ ]:
sb.calibrate_sandbox()

## Using Gempy
If you want to use gempy models to visualize in the sandbox please create or load a model and then pass it as an argument to the function.

In [ ]:
import os,sys
sys.path.append('../../../gempy')
sys.path.append('../../')
import gempy as gp
os.environ["THEANO_FLAGS"] = "mode=FAST_RUN"
import sandbox as sb
CALIBRATION_FILE = '../calibration_files/my_calibration.json'

In [ ]:
#### create example models from data_input files
data_path = '04_GempyModule/Example_Models/inputdata/'
def create_example_model(name, extent = [0,1000,0,1000,0,2300], do_sections=False, 
                         change_color=False, data_path=data_path, theano_optimizer='fast_compile'):
    
    all_models = ['Horizontal_layers', 'Recumbent_fold', 'Anticline',
             'Pinchout', 'Fault', 'Unconformity']
    
    assert name in all_models, 'possible model names are '+ str(all_models)
    geo_model = gp.create_model(name)
    
    if name == 'Horizontal_layers':
        geo_model = gp.init_data(geo_model, extent=extent,resolution=[2,2,2], 
                                path_o = data_path+"model1_orientations.csv",
                                path_i = data_path+"model1_surface_points.csv")
        if change_color:
            geo_model.surfaces.colors.change_colors({"rock2": '#9f0052', 'rock1': '#e36746', 
                                                 'basement': '#f9f871'})

        gp.map_series_to_surfaces(geo_model, {"Strat_Series": ('rock2','rock1'),
                                             "Basement_Series":('basement')})
    
    elif name == 'Recumbent_fold':
        geo_model = gp.init_data(geo_model, extent=extent,resolution=[2,2,2], 
                        path_o = data_path + "model3_orientations.csv",
                        path_i = data_path + "model3_surface_points.csv") 
        gp.map_series_to_surfaces(geo_model, {"Strat_Series": ('rock2','rock1'),
                                             "Basement_Series":('basement')})
        if change_color:
            geo_model.surfaces.colors.change_colors({"rock2": '#e36746', 'rock1': '#c0539f', 
                                                 'basement': '#006fa8'})
            
    elif name == 'Anticline':
        geo_model = gp.init_data(geo_model, extent=extent, resolution=[2,2,2], 
                                  path_o = data_path + "model2_orientations.csv",
                                  path_i = data_path + "model2_surface_points.csv")
        gp.map_series_to_surfaces(geo_model, {"Strat_Series": ('rock2','rock1'),
                                             "Basement_Series":('basement')})
        
    elif name == 'Pinchout':
        geo_model = gp.init_data(geo_model, extent=extent,resolution=[2,2,2], 
                                path_o = data_path + "model4_orientations.csv",
                                path_i = data_path + "model4_surface_points.csv")
        gp.map_series_to_surfaces(geo_model, {"Strat_Series": ('rock2','rock1'),
                                             "Basement_Series":('basement')})
        if change_color:
            geo_model.surfaces.colors.change_colors({"rock2": '#a1b455', 'rock1': '#ffbe00', 
                                                 'basement': '#006471'})
            
    elif name == 'Fault':
        geo_model = gp.init_data(geo_model, extent=extent,resolution=[2,2,2], 
                                path_o = data_path + "model5_orientations.csv",
                                path_i = data_path + "model5_surface_points.csv") 
        gp.map_series_to_surfaces(geo_model, {"Fault_Series":'fault', 
                                 "Strat_Series": ('rock2','rock1')})
        geo_model.set_is_fault(['Fault_Series'], change_color=False)
        if change_color:
            geo_model.surfaces.colors.change_colors({"rock2": '#00c2d0', 'rock1': '#a43d00', 
                                                 'basement': '#76a237', 'fault':'#000000'})
            
    elif name == 'Unconformity':
        geo_model = gp.init_data(geo_model, extent=extent, resolution=[2,2,2], 
                                path_o = data_path + "model6_orientations.csv",
                                path_i = data_path + "model6_surface_points.csv")
        
        gp.map_series_to_surfaces(geo_model, {"Strat_Series1": ('rock3'),
                                             "Strat_Series2": ('rock2','rock1'),
                                             "Basement_Series":('basement')})
        
    if do_sections:
        geo_model.grid.set_section_grid({'section'+' '+name:([0,500],[1000,500],[30,30])})

    interp_data = gp.set_interpolator(geo_model, compile_theano=True,
                                        theano_optimizer=theano_optimizer)

    _=gp.compute_model(geo_model, compute_mesh=False)
    
    if do_sections:
        gp.plot.plot_section_by_name(geo_model,'section'+' '+name, show_data=False)
        
    return geo_model

######## Load all example models

all_models = ['Horizontal_layers', 'Recumbent_fold', 'Anticline','Pinchout', 'Fault', 'Unconformity']

model_dict = {}
for model in all_models:
    model_dict.update({model: create_example_model(model)})


##### select one model to start
geo_model = model_dict['Horizontal_layers']
#model = 'Fault'
#geo_model = create_example_model(model)
geo_model

In [ ]:
##### start the server
sb.start_server(CALIBRATION_FILE, aruco_marker=True, geo_model=geo_model)